# NB16: Introduction to Zero-Knowledge Proofs [ADVANCED]

**Digital Finance - Day 04: Advanced Cryptography**

In this notebook, we explore Zero-Knowledge Proofs (ZKPs) - cryptographic protocols that allow one party (the prover) to convince another party (the verifier) that a statement is true, without revealing any information beyond the validity of the statement itself.

## Learning Objectives

By the end of this notebook, you will understand:

1. **Core ZK Properties**: Completeness, soundness, and zero-knowledge
2. **Commitment Schemes**: The building blocks of ZK proofs
3. **Interactive Proofs**: How challenge-response protocols work
4. **Fiat-Shamir Transform**: Converting interactive proofs to non-interactive
5. **Real-World Applications**: Privacy-preserving authentication and verification

**Prerequisites**: Basic understanding of hash functions and probability

## The Colorblind Friend Analogy

Imagine you have two balls: one red and one green. Your friend is colorblind and believes they're the same color.

**Your Goal**: Prove the balls are different colors without revealing which is red and which is green.

**Protocol**:
1. Your friend holds both balls behind their back
2. They show you one ball, hide it, then show you one ball again
3. You tell them if it's the same ball or if they switched

If the balls were the same color, you'd guess correctly only 50% of the time. But if they're different, you'll always be correct. After many rounds, your friend becomes convinced the colors differ - without learning which is red or green!

This is a **zero-knowledge proof**.

### Quick Primer: Hash Functions

Before we start coding, let's understand **hash functions** - they're the mathematical foundation of our zero-knowledge proofs.

**What is a hash function?**

A hash function takes any input and produces a fixed-size "fingerprint" (called a hash).

**Key Properties**:
1. **Deterministic**: Same input → same hash (always)
2. **One-way**: Easy to compute hash, impossible to reverse
3. **Avalanche effect**: Tiny input change → completely different hash
4. **Collision resistant**: Nearly impossible to find two different inputs with the same hash

**Example**:
```python
hash("hello") = "2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824"
hash("hallo") = "d3751d33f9cd5049c4af2b462735457e4d3baf130bcbb87f389e349fbaeb20b9"
                  ↑ Completely different! Just changed one letter.
```

**Why we use them in ZK proofs**:
- They act as "sealed envelopes" (commitments)
- They generate unpredictable challenges (Fiat-Shamir)
- They hide information while proving you know it

We'll use Python's `hashlib` library, which provides secure hash functions like SHA-256.

In [ ]:
# Setup: Import required libraries
import hashlib
import random
import time
import json
import matplotlib.pyplot as plt
from typing import Tuple, Dict, List, Any

# Set random seed for reproducibility
random.seed(42)

print("✓ Libraries imported successfully")
print("✓ Ready to explore Zero-Knowledge Proofs!")

---
## Section 1: Commitment Schemes

### What is a Commitment?

Imagine you want to make a prediction about tomorrow's weather, but you want to prove later that you made the prediction BEFORE seeing the actual weather.

**The Sealed Envelope Analogy**:
1. **Commit Phase**: You write "It will rain tomorrow" on paper, put it in an envelope, seal it, and give it to your friend
2. **Reveal Phase**: Tomorrow, after the weather happens, you tell your friend to open the envelope

**Key Properties**:
- **Hiding**: Your friend can't see what's inside the sealed envelope (your prediction is secret)
- **Binding**: Once sealed, you can't change what's written inside (you're locked into your prediction)

In cryptography, we use **hash functions** as our "sealed envelope"!

---

### Hash-Based Commitments

A **commitment scheme** is like a locked box: you put a value inside (commit), lock it, and can later open it to reveal the value.

**Properties**:
- **Hiding**: The commitment reveals nothing about the value
- **Binding**: Once committed, you cannot change the value

We'll implement a simple hash-based commitment scheme.

In [ ]:
class HashCommitment:
    """Simple hash-based commitment scheme."""
    
    @staticmethod
    def commit(value: str, randomness: str = None) -> Tuple[str, Dict[str, str]]:
        """Create commitment to a value.
        
        Args:
            value: The value to commit to
            randomness: Random blinding factor (generated if not provided)
            
        Returns:
            (commitment, opening) where:
            - commitment: hash that can be shared publicly
            - opening: secret information needed to verify later
        """
        if randomness is None:
            randomness = str(random.getrandbits(128))
        
        # Commitment is hash of value concatenated with randomness
        commitment = hashlib.sha256(f"{value}|{randomness}".encode()).hexdigest()
        
        # Opening contains the original value and randomness
        opening = {"value": value, "randomness": randomness}
        
        return commitment, opening
    
    @staticmethod
    def verify(commitment: str, opening: Dict[str, str]) -> bool:
        """Verify commitment matches opening.
        
        Args:
            commitment: The public commitment hash
            opening: The revealed value and randomness
            
        Returns:
            True if opening matches commitment, False otherwise
        """
        expected = hashlib.sha256(
            f"{opening['value']}|{opening['randomness']}".encode()
        ).hexdigest()
        
        return commitment == expected

print("✓ HashCommitment class defined")

In [ ]:
# Demo: Basic commitment
print("=== Basic Commitment Demo ===")
print()

secret_value = "Alice knows the password is: hunter2"
print(f"Secret value: {secret_value}")
print()

# Create commitment
commitment, opening = HashCommitment.commit(secret_value)
print(f"Commitment (public): {commitment}")
print(f"Opening (secret):    {opening}")
print()

# Verify commitment
is_valid = HashCommitment.verify(commitment, opening)
print(f"Verification: {is_valid}")
print()

# Try invalid opening
fake_opening = {"value": "wrong value", "randomness": opening["randomness"]}
is_valid_fake = HashCommitment.verify(commitment, fake_opening)
print(f"Verification with wrong value: {is_valid_fake}")

### Property 1: Hiding

The commitment reveals **nothing** about the committed value. Even if you try every possible value, the commitment looks random.

In [ ]:
# Demonstrate hiding property
print("=== Hiding Property ===")
print()
print("Commitments to different values look equally random:")
print()

values = ["heads", "tails", "Alice", "Bob", "42", "9999"]
for val in values:
    comm, _ = HashCommitment.commit(val)
    print(f"  commit('{val:5s}') = {comm[:16]}...")

print()
print("→ No pattern visible! Commitments hide the value.")

### Property 2: Binding

Once you create a commitment, you **cannot** change your mind about the value. The commitment "binds" you to your choice.

In [ ]:
# Demonstrate binding property
print("=== Binding Property ===")
print()

# Alice commits to "heads"
original_value = "heads"
commitment, opening = HashCommitment.commit(original_value)
print(f"Alice commits to: '{original_value}'")
print(f"Commitment: {commitment[:32]}...")
print()

# Later, Alice tries to claim she committed to "tails"
print("Alice tries to cheat by claiming she committed to 'tails'...")
fake_opening = {"value": "tails", "randomness": opening["randomness"]}
is_valid = HashCommitment.verify(commitment, fake_opening)
print(f"Verification result: {is_valid}")
print()

# Can Alice find different randomness that works?
print("Can Alice find different randomness that makes 'tails' work?")
print("Trying 1,000,000 random values...")
found = False
for _ in range(1_000_000):
    r = str(random.getrandbits(128))
    if HashCommitment.verify(commitment, {"value": "tails", "randomness": r}):
        found = True
        break

print(f"Found collision: {found}")
print()
print("→ Binding property holds! Alice cannot change her committed value.")

### Exercise: Commitment-Based Coin Flip

Use commitments to create a fair coin flip protocol between two parties who don't trust each other.

In [ ]:
class FairCoinFlip:
    """Fair coin flip using commitments."""
    
    @staticmethod
    def play_game(alice_choice: str, bob_choice: str, verbose: bool = True) -> str:
        """Play a fair coin flip game.
        
        Args:
            alice_choice: 'heads' or 'tails'
            bob_choice: 'heads' or 'tails'
            verbose: Print game progress
            
        Returns:
            Winner: 'Alice' or 'Bob'
        """
        if verbose:
            print("=== Fair Coin Flip Protocol ===")
            print()
        
        # Phase 1: Alice commits to her choice
        alice_commitment, alice_opening = HashCommitment.commit(alice_choice)
        if verbose:
            print(f"1. Alice commits to her choice")
            print(f"   Commitment: {alice_commitment[:32]}...")
            print()
        
        # Phase 2: Bob reveals his choice (Alice already committed, can't change)
        if verbose:
            print(f"2. Bob reveals his choice: '{bob_choice}'")
            print()
        
        # Phase 3: Alice reveals her choice
        if verbose:
            print(f"3. Alice reveals her choice: '{alice_choice}'")
        
        # Verify Alice didn't cheat
        is_valid = HashCommitment.verify(alice_commitment, alice_opening)
        if not is_valid:
            if verbose:
                print("   ❌ Alice cheated! Commitment doesn't match.")
            return "Bob (Alice cheated)"
        
        if verbose:
            print("   ✓ Commitment verified")
            print()
        
        # Determine winner
        if alice_choice == bob_choice:
            winner = "Bob"
        else:
            winner = "Alice"
        
        if verbose:
            print(f"Result: {winner} wins!")
            print()
        
        return winner

# Demo the protocol
FairCoinFlip.play_game("heads", "tails")

In [ ]:
# Run multiple games to verify fairness
print("Running 1000 fair coin flips...")
print()

alice_wins = 0
bob_wins = 0

for _ in range(1000):
    alice_choice = random.choice(["heads", "tails"])
    bob_choice = random.choice(["heads", "tails"])
    winner = FairCoinFlip.play_game(alice_choice, bob_choice, verbose=False)
    
    if winner == "Alice":
        alice_wins += 1
    else:
        bob_wins += 1

print(f"Alice wins: {alice_wins}/1000 ({alice_wins/10:.1f}%)")
print(f"Bob wins:   {bob_wins}/1000 ({bob_wins/10:.1f}%)")
print()
print("→ Protocol is fair! Each party wins ~50% of the time.")

---
## Section 2: Interactive Zero-Knowledge Proofs

Now we'll build a real zero-knowledge proof system using **graph 3-coloring**.

### What is a Graph?

Before we dive in, let's understand what a "graph" means in computer science (not the charts you see in Excel!).

**Graph = Dots connected by lines**

Think of a social network:
- **Nodes (vertices)** = People (represented as dots)
- **Edges** = Friendships connecting people (represented as lines)

Example:
```
    Alice
     / \
   Bob--Carol
```

Here, Alice is friends with Bob and Carol. Bob is friends with Alice and Carol. Carol is friends with everyone.

**Graph Coloring**: Assign a color to each person such that no two friends have the same color.

Why would we want this? Imagine scheduling shifts where friends can't work together, or assigning radio frequencies where nearby towers can't use the same frequency.

**3-coloring**: Using only 3 colors (say Red, Green, Blue) to color all nodes so no two connected nodes share a color.

---

### The Zero-Knowledge Protocol

**Problem**: Given a graph, color each node with one of 3 colors such that no two adjacent nodes have the same color.

**Scenario**: Alice has found a valid 3-coloring. She wants to prove this to Bob without revealing her coloring.

**Protocol** (one round):
1. Alice randomly permutes the colors and commits to each node's permuted color
2. Bob challenges Alice by choosing a random edge
3. Alice reveals the colors of the two nodes on that edge
4. Bob verifies: (a) commitments match, (b) colors are different

Repeat many times to achieve high confidence.

In [ ]:
class GraphColoringZKProof:
    """
    Zero-knowledge proof for graph 3-coloring.
    Proves you know a valid 3-coloring without revealing it.
    """
    
    def __init__(self, graph: Dict[str, List[str]], coloring: Dict[str, int]):
        """Initialize proof system.
        
        Args:
            graph: adjacency list {node: [neighbors]}
            coloring: {node: color} where color in {0, 1, 2}
        """
        self.graph = graph
        self.coloring = coloring
        
        # Verify coloring is valid
        for node, neighbors in graph.items():
            for neighbor in neighbors:
                if self.coloring[node] == self.coloring[neighbor]:
                    raise ValueError(f"Invalid coloring: {node} and {neighbor} have same color")
    
    def create_round(self) -> Dict[str, Any]:
        """Prover: Create commitments with random permutation.
        
        Returns:
            Round data containing commitments and secret openings
        """
        # Randomly permute the three colors
        perm = list(range(3))
        random.shuffle(perm)
        
        # Apply permutation to coloring
        permuted = {node: perm[color] for node, color in self.coloring.items()}
        
        # Commit to each node's permuted color
        commitments = {}
        openings = {}
        
        for node, color in permuted.items():
            comm, opening = HashCommitment.commit(str(color))
            commitments[node] = comm
            openings[node] = opening
        
        return {
            "commitments": commitments,
            "openings": openings,
            "permuted": permuted
        }
    
    def respond_challenge(self, round_data: Dict[str, Any], edge: Tuple[str, str]) -> Dict[str, Any]:
        """Prover: Respond to verifier's edge challenge.
        
        Args:
            round_data: Data from create_round()
            edge: The edge (u, v) challenged by verifier
            
        Returns:
            Openings for the two nodes on the edge
        """
        u, v = edge
        return {
            "u_opening": round_data["openings"][u],
            "v_opening": round_data["openings"][v]
        }
    
    @staticmethod
    def verify_response(commitments: Dict[str, str], edge: Tuple[str, str], 
                       response: Dict[str, Any]) -> bool:
        """Verifier: Check response without learning full coloring.
        
        Args:
            commitments: Public commitments from this round
            edge: The challenged edge (u, v)
            response: Prover's response with openings
            
        Returns:
            True if response is valid, False otherwise
        """
        u, v = edge
        
        # Verify commitments match openings
        if not HashCommitment.verify(commitments[u], response["u_opening"]):
            return False
        if not HashCommitment.verify(commitments[v], response["v_opening"]):
            return False
        
        # Check colors are different
        u_color = response["u_opening"]["value"]
        v_color = response["v_opening"]["value"]
        
        return u_color != v_color

print("✓ GraphColoringZKProof class defined")

In [ ]:
# Create a simple triangle graph
triangle_graph = {
    "A": ["B", "C"],
    "B": ["A", "C"],
    "C": ["A", "B"]
}

# Alice's valid 3-coloring (secret!)
alice_coloring = {
    "A": 0,  # Red
    "B": 1,  # Green
    "C": 2   # Blue
}

print("=== Graph 3-Coloring Setup ===")
print()
print("Graph (triangle):")
print("    A")
print("   / \\")
print("  B---C")
print()
print(f"Alice's secret coloring: {alice_coloring}")
print("(0=Red, 1=Green, 2=Blue)")
print()

# Create proof system
zkp = GraphColoringZKProof(triangle_graph, alice_coloring)
print("✓ Zero-knowledge proof system initialized")

In [ ]:
# Demo: Single round of the protocol
print("=== Interactive ZK Proof - Single Round ===")
print()

# Step 1: Alice creates commitments
print("Step 1: Alice creates commitments with random permutation")
round_data = zkp.create_round()
commitments = round_data["commitments"]
print()
for node, comm in commitments.items():
    print(f"  Node {node}: {comm[:32]}...")
print()

# Step 2: Bob challenges a random edge
edges = [("A", "B"), ("B", "C"), ("A", "C")]
challenged_edge = random.choice(edges)
print(f"Step 2: Bob challenges edge {challenged_edge}")
print()

# Step 3: Alice responds
print("Step 3: Alice reveals colors for challenged edge")
response = zkp.respond_challenge(round_data, challenged_edge)
u_color = response["u_opening"]["value"]
v_color = response["v_opening"]["value"]
print(f"  {challenged_edge[0]} has color: {u_color}")
print(f"  {challenged_edge[1]} has color: {v_color}")
print()

# Step 4: Bob verifies
print("Step 4: Bob verifies response")
is_valid = GraphColoringZKProof.verify_response(commitments, challenged_edge, response)
print(f"  Commitments match: ✓")
print(f"  Colors different: ✓")
print(f"  Round result: {'ACCEPT' if is_valid else 'REJECT'}")
print()

# What did Bob learn?
print("What Bob learned:")
print(f"  - Edge {challenged_edge} has different colors ✓")
print(f"  - Full coloring: Still secret! ✓")
print(f"  - Colors of other nodes: Unknown ✓")

In [ ]:
# Run multiple rounds to build confidence
def run_zk_protocol(zkp: GraphColoringZKProof, num_rounds: int = 10, 
                    verbose: bool = True) -> bool:
    """Run multiple rounds of the ZK protocol.
    
    Args:
        zkp: The ZK proof system
        num_rounds: Number of rounds to run
        verbose: Print progress
        
    Returns:
        True if all rounds pass, False otherwise
    """
    edges = [("A", "B"), ("B", "C"), ("A", "C")]
    
    if verbose:
        print(f"=== Running {num_rounds} Rounds ===")
        print()
    
    for round_num in range(1, num_rounds + 1):
        # Prover creates commitments
        round_data = zkp.create_round()
        commitments = round_data["commitments"]
        
        # Verifier challenges random edge
        edge = random.choice(edges)
        
        # Prover responds
        response = zkp.respond_challenge(round_data, edge)
        
        # Verifier checks
        is_valid = GraphColoringZKProof.verify_response(commitments, edge, response)
        
        if verbose:
            status = "✓ PASS" if is_valid else "✗ FAIL"
            print(f"Round {round_num:2d}: Edge {edge} → {status}")
        
        if not is_valid:
            return False
    
    if verbose:
        print()
        print(f"All {num_rounds} rounds passed!")
        print(f"Verifier's confidence: {(1 - (1/3)**num_rounds) * 100:.10f}%")
    
    return True

# Run 20 rounds
run_zk_protocol(zkp, num_rounds=20)

### Soundness Analysis

**Question**: If Alice is cheating (doesn't have a valid coloring), what's the probability she gets caught?

- In each round, Bob checks 1 of 3 edges
- If the coloring is invalid, at least 1 edge must have the same color on both ends
- Probability of choosing the bad edge: ≥ 1/3
- Probability Alice survives one round while cheating: ≤ 2/3
- Probability Alice survives n rounds while cheating: ≤ (2/3)^n

After 20 rounds: (2/3)^20 ≈ 0.026% chance of undetected cheating!

In [ ]:
# Visualize confidence vs number of rounds
rounds = list(range(1, 31))
confidence = [1 - (2/3)**n for n in rounds]

plt.figure(figsize=(10, 6))
plt.plot(rounds, [c * 100 for c in confidence], linewidth=2, color='#2E86AB')
plt.axhline(y=99, color='red', linestyle='--', alpha=0.5, label='99% confidence')
plt.axhline(y=99.9, color='darkred', linestyle='--', alpha=0.5, label='99.9% confidence')

plt.xlabel('Number of Rounds', fontsize=12)
plt.ylabel('Verifier Confidence (%)', fontsize=12)
plt.title('ZK Proof Confidence vs Number of Rounds', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()

# Annotations
plt.annotate('10 rounds: 98.3%', xy=(10, 98.3), xytext=(12, 90),
            arrowprops=dict(arrowstyle='->', color='gray'),
            fontsize=10)
plt.annotate('20 rounds: 99.97%', xy=(20, 99.97), xytext=(22, 95),
            arrowprops=dict(arrowstyle='->', color='gray'),
            fontsize=10)

plt.tight_layout()
plt.show()

print("Key takeaway: Confidence grows exponentially with rounds!")
print(f"  5 rounds:  {confidence[4] * 100:.2f}%")
print(f" 10 rounds:  {confidence[9] * 100:.2f}%")
print(f" 20 rounds:  {confidence[19] * 100:.4f}%")
print(f" 30 rounds:  {confidence[29] * 100:.8f}%")

### Testing Against a Cheating Prover

Let's see what happens if Alice tries to cheat with an invalid coloring.

In [ ]:
print("=== Testing Against Invalid Coloring ===")
print()

# Invalid coloring: A and B have the same color
invalid_coloring = {
    "A": 0,  # Red
    "B": 0,  # Red (same as A! Invalid!)
    "C": 1   # Green
}

print(f"Alice's invalid coloring: {invalid_coloring}")
print("Problem: Nodes A and B are adjacent but have the same color!")
print()

# Try to create a proof system (this will work, but proofs will fail)
# We need to bypass the validation for demonstration
cheating_zkp = GraphColoringZKProof.__new__(GraphColoringZKProof)
cheating_zkp.graph = triangle_graph
cheating_zkp.coloring = invalid_coloring

print("Running 20 rounds with invalid coloring...")
print()

edges = [("A", "B"), ("B", "C"), ("A", "C")]
caught = False
caught_round = 0

for round_num in range(1, 21):
    round_data = cheating_zkp.create_round()
    commitments = round_data["commitments"]
    edge = random.choice(edges)
    response = cheating_zkp.respond_challenge(round_data, edge)
    is_valid = GraphColoringZKProof.verify_response(commitments, edge, response)
    
    status = "✓ PASS" if is_valid else "✗ FAIL (CAUGHT!)"  
    print(f"Round {round_num:2d}: Edge {edge} → {status}")
    
    if not is_valid:
        caught = True
        caught_round = round_num
        break

print()
if caught:
    print(f"🚨 Alice caught cheating in round {caught_round}!")
else:
    print("Alice got lucky and wasn't caught (happens ~2.6% of the time)")

---
## Section 3: Non-Interactive ZK Proofs (Fiat-Shamir)

### The Problem with Interactive Proofs

Our graph coloring proof requires back-and-forth communication:
1. Alice sends commitments → 
2. ← Bob sends challenge
3. Alice sends response →

This is called **interactive** - like having a conversation.

**Problem**: What if Bob is offline? Or Alice wants to post a proof publicly for anyone to verify?

### The Fiat-Shamir Transform: From Conversation to Single Message

**Clever Trick**: Instead of Bob choosing the challenge, use a hash function to generate it!

**How it works**:
- **Before**: Bob picks random challenge after seeing commitments
- **After**: Challenge = `hash(commitments)`

**Why this is secure**:
1. Alice creates commitments FIRST
2. Challenge is determined by hashing those commitments
3. Alice can't change commitments after seeing challenge (binding property!)
4. Alice can't predict challenge before creating commitments (hash is unpredictable!)

**Result**: Alice can create the entire proof (commitments + response) in one shot, and anyone can verify it later!

**Before (Interactive)**:
```
Alice ← needs Bob → Bob
  Multiple rounds
  Real-time communication
```

**After (Non-Interactive)**:
```
Alice → single proof → [anyone can verify]
  One message
  Post publicly
  Verify anytime
```

In [ ]:
def fiat_shamir_challenge(graph: Dict[str, List[str]], 
                         commitments: Dict[str, str]) -> Tuple[str, str]:
    """Generate deterministic challenge using hash (Fiat-Shamir heuristic).
    
    WHY THIS IS SECURE:
    
    1. Alice creates commitments BEFORE the challenge exists
    2. Challenge is computed from hash(commitments)
    3. Alice can't predict hash output (hash functions are one-way)
    4. Alice can't change commitments after seeing challenge (binding property)
    5. Result: Alice can't cheat by choosing favorable challenges!
    
    This transforms interactive proofs (back-and-forth conversation) into
    non-interactive proofs (single message that anyone can verify).
    
    Args:
        graph: The graph structure
        commitments: Alice's commitments for this round
        
    Returns:
        Challenged edge (u, v)
    """
    # Create deterministic string from all commitments
    combined = json.dumps(commitments, sort_keys=True)
    
    # Hash to get "random" challenge
    # The hash acts as an unpredictable random oracle:
    # - Same commitments → same challenge (deterministic)
    # - Different commitments → completely different challenge (unpredictable)
    hash_val = hashlib.sha256(combined.encode()).hexdigest()
    
    # Use hash to select edge
    edges = []
    for u in graph:
        for v in graph[u]:
            if u < v:  # Add each edge only once
                edges.append((u, v))
    
    idx = int(hash_val[:8], 16) % len(edges)
    return edges[idx]

print("✓ Fiat-Shamir transform implemented")

In [ ]:
# Demo: Non-interactive proof
print("=== Non-Interactive ZK Proof (Fiat-Shamir) ===")
print()
print("Alice creates a non-interactive proof:")
print()

# Alice generates commitments
round_data = zkp.create_round()
commitments = round_data["commitments"]

print("1. Alice creates commitments:")
for node, comm in commitments.items():
    print(f"   Node {node}: {comm[:32]}...")
print()

# Challenge is deterministically derived from commitments
challenge_edge = fiat_shamir_challenge(triangle_graph, commitments)
print(f"2. Challenge (from hash): {challenge_edge}")
print()

# Alice includes response in the proof
response = zkp.respond_challenge(round_data, challenge_edge)
print("3. Alice includes openings for challenged edge")
print()

# Now Alice sends the entire proof: (commitments, response)
# Bob can verify without any interaction!
proof = {
    "commitments": commitments,
    "response": response
}

print("Bob verifies the proof:")
# Bob re-derives the challenge
bob_challenge = fiat_shamir_challenge(triangle_graph, proof["commitments"])
print(f"  - Derived challenge: {bob_challenge}")

# Bob verifies the response
is_valid = GraphColoringZKProof.verify_response(
    proof["commitments"], 
    bob_challenge, 
    proof["response"]
)
print(f"  - Verification: {'✓ PASS' if is_valid else '✗ FAIL'}")
print()

print("✓ Non-interactive proof completed successfully!")

In [ ]:
# Show that same commitments always give same challenge
print("=== Determinism of Fiat-Shamir ===")
print()
print("Same commitments → Same challenge:")
print()

# Generate commitments
round_data = zkp.create_round()
commitments = round_data["commitments"]

# Derive challenge multiple times
for i in range(5):
    challenge = fiat_shamir_challenge(triangle_graph, commitments)
    print(f"  Attempt {i+1}: {challenge}")

print()
print("Different commitments → Different challenges:")
print()

for i in range(5):
    round_data = zkp.create_round()
    commitments = round_data["commitments"]
    challenge = fiat_shamir_challenge(triangle_graph, commitments)
    print(f"  Round {i+1}: {challenge}")

print()
print("→ Fiat-Shamir is deterministic but unpredictable!")

### Why Fiat-Shamir Works

**Security intuition**:
1. Alice creates commitments BEFORE knowing the challenge
2. Challenge is determined by hash of commitments
3. Alice cannot change commitments after seeing challenge (binding property)
4. Alice cannot predict challenge before committing (hash is one-way)

This transforms an interactive proof into a non-interactive one that can be verified by anyone, anytime!

---
## Section 4: ZK Applications

Now let's build practical applications using zero-knowledge proofs.

In [ ]:
class AgeVerification:
    """Prove age >= threshold without revealing exact age."""
    
    def __init__(self, birth_year: int):
        """Initialize with secret birth year.
        
        Args:
            birth_year: Year of birth (kept secret)
        """
        self.birth_year = birth_year
    
    def prove_adult(self, current_year: int = 2025, threshold: int = 18) -> Dict[str, Any]:
        """Create proof that age >= threshold.
        
        Args:
            current_year: Current year
            threshold: Minimum age requirement
            
        Returns:
            ZK proof object
        """
        age = current_year - self.birth_year
        is_adult = age >= threshold
        
        # Commitment to birth year
        comm, opening = HashCommitment.commit(str(self.birth_year))
        
        # In a real ZK system, we'd have a cryptographic proof here
        # For this demo, we simulate the proof structure
        return {
            "commitment": comm,
            "statement": f"Prover's age >= {threshold}",
            "result": is_adult,
            "current_year": current_year,
            "threshold": threshold,
            # For demo purposes only - real ZK wouldn't include this!
            "_opening_for_demo": opening
        }
    
    @staticmethod
    def verify_proof(proof: Dict[str, Any]) -> bool:
        """Verify age proof.
        
        In a real system, this would use cryptographic verification.
        For demo, we just check the commitment is valid.
        
        Args:
            proof: The proof object
            
        Returns:
            True if proof is valid, False otherwise
        """
        # Verify commitment (in real ZK, we'd verify the cryptographic proof)
        is_commitment_valid = HashCommitment.verify(
            proof["commitment"], 
            proof["_opening_for_demo"]
        )
        
        if not is_commitment_valid:
            return False
        
        # In real ZK: verify cryptographic proof of age >= threshold
        # For demo: we trust the result in the proof
        return proof["result"]

print("✓ AgeVerification class defined")

In [ ]:
# Demo: Age verification
print("=== Age Verification Demo ===")
print()

# Alice was born in 1995
alice = AgeVerification(birth_year=1995)
print("Alice wants to prove she's an adult without revealing her exact age.")
print()

# Generate proof
proof = alice.prove_adult(current_year=2025, threshold=18)
print("Alice's proof:")
print(f"  Commitment: {proof['commitment'][:32]}...")
print(f"  Statement: {proof['statement']}")
print()

# Verifier checks
is_valid = AgeVerification.verify_proof(proof)
print(f"Verification result: {is_valid}")
print()

print("What the verifier learned:")
print("  ✓ Alice is at least 18 years old")
print("  ✗ Alice's exact age: UNKNOWN")
print("  ✗ Alice's birth year: UNKNOWN")
print()

# Try with someone underage
print("---")
print()
bob = AgeVerification(birth_year=2010)
print("Bob (born 2010) tries to prove he's an adult...")
proof = bob.prove_adult(current_year=2025, threshold=18)
is_valid = AgeVerification.verify_proof(proof)
print(f"Verification result: {is_valid}")
print("Bob's proof fails (as expected).")

In [ ]:
class BalanceProof:
    """Prove balance >= amount without revealing exact balance."""
    
    def __init__(self, balance: float):
        """Initialize with secret balance.
        
        Args:
            balance: Account balance (kept secret)
        """
        self.balance = balance
    
    def prove_sufficient_funds(self, required_amount: float) -> Dict[str, Any]:
        """Create proof that balance >= required_amount.
        
        Args:
            required_amount: Minimum required balance
            
        Returns:
            ZK proof object
        """
        has_funds = self.balance >= required_amount
        
        # Commitment to balance
        comm, opening = HashCommitment.commit(str(self.balance))
        
        return {
            "commitment": comm,
            "statement": f"Balance >= {required_amount:.2f}",
            "result": has_funds,
            "required_amount": required_amount,
            "_opening_for_demo": opening
        }
    
    @staticmethod
    def verify_proof(proof: Dict[str, Any]) -> bool:
        """Verify balance proof."""
        is_commitment_valid = HashCommitment.verify(
            proof["commitment"],
            proof["_opening_for_demo"]
        )
        
        return is_commitment_valid and proof["result"]

print("✓ BalanceProof class defined")

In [ ]:
# Demo: Balance proof
print("=== Sufficient Funds Proof Demo ===")
print()

# Alice wants to buy something for $500
alice_account = BalanceProof(balance=1250.75)
print("Scenario: Alice wants to buy an item for $500.")
print("She needs to prove she has sufficient funds without revealing her balance.")
print()

# Generate proof
proof = alice_account.prove_sufficient_funds(required_amount=500.00)
print("Alice's proof:")
print(f"  Commitment: {proof['commitment'][:32]}...")
print(f"  Statement: {proof['statement']}")
print()

# Merchant verifies
is_valid = BalanceProof.verify_proof(proof)
print(f"Merchant verification: {'✓ APPROVED' if is_valid else '✗ REJECTED'}")
print()

print("What the merchant learned:")
print("  ✓ Alice can afford the $500 item")
print("  ✗ Alice's exact balance: UNKNOWN")
print("  ✗ How much extra Alice has: UNKNOWN")
print()

# Privacy comparison
print("---")
print()
print("Privacy comparison:")
print()
print("Traditional approach:")
print("  Merchant sees: Full account balance ($1250.75)")
print("  Privacy: ✗ Poor")
print()
print("Zero-knowledge approach:")
print("  Merchant sees: Has ≥ $500 (nothing else)")
print("  Privacy: ✓ Excellent")

### Real-World ZK Applications

**Privacy-Preserving Authentication**:
- Prove you know a password without sending it
- Prove you're a member of a group without revealing which member

**Blockchain & Cryptocurrency**:
- **Zcash**: Shielded transactions hiding sender, receiver, and amount
- **zk-SNARKs**: Succinct non-interactive proofs used in many systems
- **zk-Rollups**: Ethereum scaling solutions (Polygon, zkSync)

**Financial Privacy**:
- Prove creditworthiness without revealing financial details
- Prove tax compliance without revealing transactions
- Private voting in DAOs

**Identity Verification**:
- Prove age without revealing birthdate
- Prove citizenship without revealing ID number
- Prove qualifications without revealing transcripts

---
## Section 5: Challenge Exercises

Now it's your turn to apply what you've learned!

### Challenge 1: Enhanced Commitment Scheme ⭐⭐

**Difficulty**: Intermediate

**Hint**: Start simple! If this seems hard, try implementing a basic commitment with a single hash first, then extend it.

Implement a Pedersen-like commitment scheme (simplified version).

**Requirements**:
- Use two hash functions instead of one
- Commitment: `hash1(value || r1) XOR hash2(r2)`
- Opening: `(value, r1, r2)`
- Implement `commit()` and `verify()` methods

In [ ]:
class PedersenLikeCommitment:
    """Enhanced commitment scheme inspired by Pedersen commitments."""
    
    @staticmethod
    def commit(value: str, r1: str = None, r2: str = None) -> Tuple[str, Dict[str, str]]:
        """Create commitment using two hash functions.
        
        TODO: Implement this method
        Hint: Use sha256 and sha3_256 (from hashlib)
        """
        pass
    
    @staticmethod
    def verify(commitment: str, opening: Dict[str, str]) -> bool:
        """Verify commitment.
        
        TODO: Implement this method
        """
        pass

# Test your implementation
# comm, opening = PedersenLikeCommitment.commit("secret_value")
# print(f"Commitment: {comm}")
# print(f"Valid: {PedersenLikeCommitment.verify(comm, opening)}")

### Challenge 2: Set Membership Proof ⭐⭐⭐

**Difficulty**: Advanced

**Starter Hint**: Review the graph coloring protocol above - the structure is similar!

Create a ZK proof that proves an element is in a set without revealing which element.

**Example**: Prove you're on a whitelist without revealing your identity.

**Approach**:
1. Commit to your element
2. Verifier challenges with random subset
3. Reveal if your element is in the challenged subset
4. Repeat multiple rounds

In [ ]:
class SetMembershipProof:
    """Prove membership in a set without revealing which element."""
    
    def __init__(self, element: str, full_set: List[str]):
        """Initialize with secret element and public set.
        
        Args:
            element: Your secret element (must be in full_set)
            full_set: The complete set
        """
        if element not in full_set:
            raise ValueError("Element must be in the set")
        self.element = element
        self.full_set = full_set
    
    def create_round(self) -> Dict[str, Any]:
        """Create commitment for one round.
        
        TODO: Implement this method
        Hint: Commit to a permuted version of your element
        """
        pass
    
    def respond_challenge(self, round_data: Dict[str, Any], 
                         challenged_subset: List[str]) -> Dict[str, Any]:
        """Respond to challenge.
        
        TODO: Implement this method
        Return whether your element is in the challenged subset
        """
        pass
    
    @staticmethod
    def verify_response(commitment: str, challenged_subset: List[str],
                       response: Dict[str, Any]) -> bool:
        """Verify response.
        
        TODO: Implement this method
        """
        pass

# Example usage:
# whitelist = ["alice", "bob", "charlie", "diana", "eve"]
# proof = SetMembershipProof("alice", whitelist)
# # Run multiple rounds to prove membership

### Challenge 3: Private Voting System ⭐⭐⭐⭐

**Difficulty**: Expert (Capstone Project)

**Starter Tip**: Begin by implementing just the commitment and nullifier parts. Add the ZK proof components after that works.

Design a voting scheme where:
- Voters prove they're eligible without revealing their identity
- Votes are private but verifiable
- Results are publicly auditable

**Requirements**:
1. Use commitments for votes
2. Voters prove eligibility (set membership)
3. Prevent double voting
4. Allow public verification of results

In [ ]:
class PrivateVotingSystem:
    """Private voting using zero-knowledge proofs."""
    
    def __init__(self, eligible_voters: List[str], candidates: List[str]):
        """Initialize voting system.
        
        Args:
            eligible_voters: List of eligible voter IDs
            candidates: List of candidate names
        """
        self.eligible_voters = eligible_voters
        self.candidates = candidates
        self.votes = []  # List of committed votes
        self.nullifiers = set()  # Prevent double voting
    
    def cast_vote(self, voter_id: str, candidate: str) -> Dict[str, Any]:
        """Cast a private vote.
        
        TODO: Implement this method
        Should:
        1. Verify voter is eligible (ZK proof)
        2. Create commitment to vote
        3. Create nullifier to prevent double voting
        4. Store vote
        """
        pass
    
    def tally_votes(self) -> Dict[str, int]:
        """Count votes (after revealing phase).
        
        TODO: Implement this method
        In practice, there would be a reveal phase where
        commitments are opened in a privacy-preserving way
        """
        pass
    
    def verify_election(self) -> bool:
        """Verify election integrity.
        
        TODO: Implement this method
        Check that all votes are valid and no double voting occurred
        """
        pass

# Example usage:
# voters = ["voter1", "voter2", "voter3", "voter4", "voter5"]
# candidates = ["Alice", "Bob", "Charlie"]
# election = PrivateVotingSystem(voters, candidates)
# 
# # Each voter casts their vote
# election.cast_vote("voter1", "Alice")
# # ...
#
# # Tally results
# results = election.tally_votes()
# print(results)

---
## Summary

### Key Concepts

1. **Zero-Knowledge Proofs**: Prove statements without revealing information
   - **Completeness**: True statements can be proven
   - **Soundness**: False statements cannot be proven
   - **Zero-knowledge**: Verifier learns nothing beyond statement validity

2. **Commitment Schemes**: Building blocks for ZK proofs
   - **Hiding**: Commitment reveals nothing about value
   - **Binding**: Cannot change committed value

3. **Interactive Proofs**: Challenge-response protocols
   - Verifier confidence grows exponentially with rounds
   - Graph coloring example demonstrates core principles

4. **Fiat-Shamir Transform**: Make proofs non-interactive
   - Hash function acts as "random oracle"
   - Enables practical deployments

5. **Real Applications**: Privacy-preserving systems
   - Age verification
   - Balance proofs
   - Private voting
   - Blockchain privacy (Zcash, zk-SNARKs)

### Further Learning

- **zk-SNARKs**: Succinct Non-Interactive Arguments of Knowledge
- **zk-STARKs**: Scalable, Transparent Arguments of Knowledge
- **Bulletproofs**: Efficient range proofs
- **Circom & SnarkJS**: Build ZK circuits
- **Zcash, Polygon zkEVM, zkSync**: Production ZK systems

### Next Steps

1. Complete the challenge exercises
2. Explore zk-SNARK libraries (libsnark, bellman)
3. Study Zcash's cryptographic construction
4. Build a simple ZK application

**Remember**: ZK proofs are powerful but complex. Production systems require careful cryptographic engineering!